# Etudiantes
Lina Berroug
et
Jessica Karega

In [ ]:
#IMPORTATION DE MODULES

import json
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from textblob import TextBlob
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import os
import gradio as gr



In [ ]:
# Télécharger les stopwords en français si ce n'est pas déjà fait
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

In [ ]:
with open('test2.json', 'r', encoding='utf-8') as f:
    donne = json.load(f)

# Introduction
Ce notebook analyse les tweets d'un fichier JSON pour extraire des informations et les exploiter grâce à un dataFrame

Nous utiliserons les bibliothèques suivantes :
- `json` : pour manipuler les fichiers JSON
- `pandas` : pour analyser les données tabulaires
- `re` : pour extraire ou supprimer des expressions régulière d'un tweet
- `TextBlob` : pour l'analyse de sentiments
- `matplotlib` : pour visualiser sous forme de graphiques les données


### Diagramme des étapes de traitement
Schéma qui montre les étapes suivies pour le projet :

<img src="diagramme.png" alt="Description de l'image" style="width:40%; height:auto;">




# <u>**PARTIE I** : Création de la classe Tweet et sa soumission dans la Landing Zone</u>

## Création de la classe Tweet
-  Nettoyer le texte brut.


## Création de la classe ZA
- Enrichir et structurer les données nettoyées.

    ### 1. Méthode : Extraction_Hashtags 
    - on utilise une regex pour extraire des hashtags standards 
    ### 2. Méthode : Extraction_Mentions 
    - on utilise une regex pour renvoyer toutes les correspondances d'un motif (mentions @) sous forme de liste.
    ### 3. Méthode : Extraction_Sentiments
    - on utilise Textblob pour déterminer le sentiment d'un texte en renvoyant une polarité (positive, neutre, négative) et une subjectivité.
    ### 4. Méthode : Id_Topic


## Création Zone d'Atterissage




In [ ]:
#PARTIE : CREATION DU TWEET + SOUMISSION DU TWEET DANS LANDING ZONE

#Création d'une Classe Tweet à laquelle seront associées les attributs suivant : auteurs, hashtag, mentions, sentiments, (topics???)

class Tweet: #Plan/Modèle
    def __init__(self, contenu): #Méthode (assisstant qui aide à donner à chaque tweet ses attributs)
        self.contenu = contenu

    def Nettoyage_Tweet(self):
        """Fonction qui supprime les caractères spéciaux d'un tweet"""
        enlever_speciaux = re.sub(r'[^a-zA-Z0-9\s@#]', '', self.contenu)
        return enlever_speciaux

class ZA:
    def __init__(self, id, auteur, texte): #Méthode (assisstant qui aide à donner à chaque tweet ses attributs)
        self.id = id
        self.auteur = auteur
        self.text = texte
        self.hashtags = self.Extraction_Hashtags()
        self.mentions = self.Extraction_Mentions()
        self.sentiment = self.Extraction_Sentiments()
        self.topic_words = self.Id_Topic()

    
    def Extraction_Sentiments(self): 
        """Extrait le sentiment du texte"""
        blob = TextBlob(self.text)
        polarite =  blob.sentiment.polarity
        if polarite > 0:
            sentiment = 'positif'
        elif polarite < 0:
            sentiment = 'négatif'
        elif polarite == 0:
            sentiment = 'neutre'
        return sentiment
    
    def Extraction_Hashtags(self):  #A Faire 
        """"""
        hashtags = re.findall(r'#\w+', self.text)
        return hashtags
    
    def Extraction_Mentions(self):  #A Faire
        """Fonction qui extrait les mentions de personnes dans un tweet"""
        mentions = re.findall(r'@[A-Za-z0-9_]+', self.text)

        return mentions


    def Id_Topic(self):
        
        # Fonction pour nettoyer le texte
        def traitement_texte(text):
            text = text.lower()  # Convertir en minuscule
            text = re.sub(r'[^a-z\s]', '', text)  # Supprimer les caractères spéciaux
            return text

        def get_stopwords():
            return stopwords.words('french')

        stopwords_list = get_stopwords()
        tweets = [infos["text"] for infos in donne]

        # Nettoyer les textes des tweets
        texte_traite = [traitement_texte(tweet) for tweet in tweets]

        # Initialiser et entraîner le vecteur TF-IDF
        vecteur = TfidfVectorizer(stop_words=stopwords_list)
        a = vecteur.fit_transform(texte_traite)

        # Entraîner le modèle LDA
        lda = LatentDirichletAllocation(n_components=20, random_state=42)
        lda.fit(a)

        # Récupérer les topics
        topics = []
        words = vecteur.get_feature_names_out()
        for i, topic_weights in enumerate(lda.components_):
            top_indices = topic_weights.argsort()[:-3:-1]  # Prendre les 2 mots les plus importants
            top_words = [words[i] for i in top_indices]
            topics.append(top_words)

        # Prédire le topic pour le tweet donné
        tweet_traite = traitement_texte(self.text)
        vecteur_tweet = vecteur.transform([tweet_traite])
        probabilities = lda.transform(vecteur_tweet)
        topic_idx = probabilities.argmax()  # Trouver l'indice du topic avec la probabilité la plus élevée
        topic_words = topics[topic_idx]  # Mots principaux du topic

        return topic_words


def Zone_Atterisssage (Fichier):
        """Stockage du tweet dans le fichier JSON zone d'atterissage"""
        ZA_liste=[]

        repertoire_courant = os.getcwd()
        fichier_entrant = os.path.join(repertoire_courant, Fichier)
        fichier_sortant = os.path.join(repertoire_courant, 'outputs', 'ZA.json')
        
        with open (fichier_entrant,'r',encoding='utf-8') as f:
            donnee= json.load(f)
            
        for infos_tweets in donnee:
            id = infos_tweets.get("id")
            auteur = infos_tweets.get("author_id")
            contenu = infos_tweets.get("text")

            tweet = Tweet(contenu)
            contenu_nettoye = tweet.Nettoyage_Tweet()
    

            tweet_analyse = ZA(id, auteur, contenu_nettoye)


            dict_tweet = {
                "id": tweet_analyse.id,
                "auteur": tweet_analyse.auteur,
                "tweet": contenu_nettoye,
                "texte": tweet_analyse.text,
                "hashtags" : tweet_analyse.hashtags,
                "mentions" : tweet_analyse.mentions,
                "sentiment" : tweet_analyse.sentiment,
                "topic" : tweet_analyse.topic_words
            }
            ZA_liste.append(dict_tweet)

        
        os.makedirs(os.path.dirname(fichier_sortant), exist_ok=True)
        with open(fichier_sortant,'w', encoding='utf-8') as f:
            json.dump(ZA_liste, f, indent=4, ensure_ascii=False)
            
if __name__ == "__main__":
    Zone_Atterisssage('test2.json')



# <u>**PARTIE II** : Data Frame et Visualisation des données</u>

## Création de la DF
- structure qui organise les données en tables bidimensionnelles de lignes et de colonnes, comparables à une feuille de calcul


## Fonction d'analyses avancées
        - Top K éléments : 
        - Publications_par_hashtag : 
        - Publications_par_utilisateurs : 
        - publications_par_topic : 
        - tweets_par_utilisateur : 


## Fonction de Visualisation sur Maplotlib et Gradio


In [15]:

# Vos fonctions d'analyse
def CreerDataFrame(fichier_json):
    df  = pd.read_json(fichier_json)
    return df

def Top_K_element(df, colonne, k=3):
    top_k = df[colonne].explode().value_counts().head(k)
    element= top_k.index.tolist()
    counts = top_k.values.tolist()
    return element, counts

def publications_par_hashtag(df):
    hashtags_explodes = df['hashtags'].explode()
    hashtag_counts = hashtags_explodes.value_counts()

    hashtags = hashtag_counts.index.tolist()
    counts = hashtag_counts.values.tolist()
    return hashtags, counts

def publications_par_utilisateurs(df):
    utilisateur_counts = df['auteur'].value_counts()
    utilisateur = utilisateur_counts.index.tolist()  
    counts = utilisateur_counts.values.tolist()   
    return utilisateur, counts

def publications_par_topic(df):
    topics_counts = df['topics'].value_counts()
    topic = topics_counts.index.tolist()  
    counts = topics_counts.values.tolist()   
    return topic , counts

def tweets_par_utilisateur(df, utilisateur):
    filtre = df[df['auteur'] == utilisateur]
    tweets = filtre['contenu'].tolist() 
    utilisateurs = filtre['auteur'].tolist() 
    return utilisateurs, tweets

# Fonctions de visualisation modifiées pour retourner des figures
def nbr_hashtag(df):
    hashtags, counts = publications_par_hashtag(df)
    fig, ax = plt.subplots()
    ax.bar(hashtags, counts, color = 'blue')
    ax.set_title("Nombre de publication par hashtag")
    ax.set_xlabel("Hashtags")
    ax.set_ylabel("Fréquence")
    plt.xticks(rotation=50, ha='right')
    plt.tight_layout()
    return fig

def nbr_utilisateur(df):
    utilisateur, counts = publications_par_utilisateurs(df)
    utilisateur = [str(u) for u in utilisateur]

    fig, ax = plt.subplots()
    ax.bar(utilisateur, counts, color = 'red')
    ax.set_title("Nombre de publication par utilisateur")
    ax.set_xlabel("utilisateurs")
    ax.set_ylabel("Fréquence")
    plt.xticks(rotation=50, ha='right')
    plt.tight_layout()
    return fig

def nbr_topic(df):
    topic, counts = publications_par_topic(df)
    fig, ax = plt.subplots()
    ax.bar(topic, counts, color = 'green')
    ax.set_title("Nombre de publication par topic")
    ax.set_xlabel("topics")
    ax.set_ylabel("Fréquence")
    plt.xticks(rotation=50, ha='right')
    plt.tight_layout()
    return fig

def top_hashtags(df, k):
    element, counts = Top_K_element(df, 'hashtags', k)
    fig, ax = plt.subplots()
    ax.bar(element, counts, color = 'purple')
    ax.set_title("Top Hashtags")
    ax.set_xlabel(f"Top {k} hashtags")
    ax.set_ylabel("Fréquence")
    plt.xticks(rotation=50, ha='right')
    plt.tight_layout()
    return fig

def top_auteur(df, k):
    element, counts = Top_K_element(df, 'auteur', k)
    element = [str(e) for e in element]
    fig, ax = plt.subplots()
    ax.bar(element, counts, color='orange')
    ax.set_title(f"Top {k} utilisateurs par nombre de publications")
    ax.set_xlabel("Utilisateur")
    ax.set_ylabel("Fréquence")
    plt.xticks(rotation=50, ha='right')
    plt.tight_layout()
    return fig

def top_mentions(df, k):
    element, counts = Top_K_element(df, 'mentions', k)
    fig, ax = plt.subplots()
    ax.bar(element, counts, color = 'blue')
    ax.set_title("Top mentions")
    ax.set_xlabel('mentions')
    ax.set_ylabel("Fréquence")
    plt.xticks(rotation=50, ha='right')
    plt.tight_layout()
    return fig

def utilisateurs_mentions_hashtag(df, hashtag):
    tweets = df[df['hashtags'].apply(lambda hashtags: hashtag in hashtags if isinstance(hashtags, list) else False)]
    utilisateurs_count = tweets['auteur'].value_counts()
    utilisateurs = [str(u) for u in utilisateurs_count.index]

    fig, ax = plt.subplots()
    ax.bar(utilisateurs, utilisateurs_count.values, color='green')
    ax.set_title(f"Nombre de tweets mentionnant {hashtag}")
    ax.set_xlabel("Utilisateurs")
    ax.set_ylabel("Nombre de tweets")
    plt.xticks(rotation=50, ha='right')
    plt.tight_layout()
    return fig

def utilisateur_mentions_utilisateur(df, mention):
    tweets = df[df['mentions'].apply(lambda mentions: mention in mentions if isinstance(mentions,list) else False )]
    utilisateurs_count = tweets['auteur'].value_counts()
    utilisateurs = [str(u) for u in utilisateurs_count.index]

    fig, ax = plt.subplots()
    ax.bar(utilisateurs, utilisateurs_count.values, color='green')
    ax.set_title(f"Nombre de tweets mentionnant {mention}")
    ax.set_xlabel("Utilisateurs")
    ax.set_ylabel("Nombre de tweets")
    plt.xticks(rotation=50, ha='right')
    plt.tight_layout()
    return fig

def utilisateurs_mentionnes_par_utilisateur(df, utilisateur):
    df['auteur'] = df['auteur'].astype(str)
    tweets_utilisateur = df[df['auteur'] == utilisateur]

    if tweets_utilisateur.empty:
        return None

    mentions = tweets_utilisateur['mentions'].explode()

    if mentions.empty:
        return None

    mentions_count = mentions.value_counts()
    mentions_index = [str(m) for m in mentions_count.index]

    fig, ax = plt.subplots()
    ax.bar(mentions_index, mentions_count.values, color='purple')
    ax.set_title(f"Utilisateurs mentionnés par '{utilisateur}'")
    ax.set_xlabel("Utilisateurs mentionnés")
    ax.set_ylabel("Nombre de mentions")
    plt.xticks(rotation=50, ha='right')
    plt.tight_layout()
    return fig

# Fonctions Gradio
def charger_df(fichier):
    df = CreerDataFrame(fichier.name)
    return df

def afficher_nbr_utilisateur(file):
    df = charger_df(file)
    return nbr_utilisateur(df)

def afficher_nbr_hashtag(file):
    df = charger_df(file)
    return nbr_hashtag(df)

def afficher_nbr_topic(file):
    df = charger_df(file)
    return nbr_topic(df)

def afficher_top_hashtags(file, k):
    df = charger_df(file)
    return top_hashtags(df, int(k))

def afficher_top_auteur(file, k):
    df = charger_df(file)
    return top_auteur(df, int(k))

def afficher_top_mentions(file, k):
    df = charger_df(file)
    return top_mentions(df, int(k))

def afficher_utilisateurs_mentions_hashtag(file, hashtag):
    df = charger_df(file)
    return utilisateurs_mentions_hashtag(df, hashtag)

def afficher_utilisateur_mentions_utilisateur(file, mention):
    df = charger_df(file)
    return utilisateur_mentions_utilisateur(df, mention)

def afficher_utilisateurs_mentionnes_par_utilisateur(file, utilisateur):
    df = charger_df(file)
    fig = utilisateurs_mentionnes_par_utilisateur(df, utilisateur)
    if fig is None:
        return "Aucune donnée"
    return fig




with gr.Blocks() as demo:
    gr.Markdown("# Interface d'Analyse")

    with gr.Tab("Nombre de publications par utilisateur"):
        file_input = gr.File(label="Charger ZA.json")
        bouton_utilisateur = gr.Button("Afficher")
        out_plot_utilisateur = gr.Plot()
        bouton_utilisateur.click(afficher_nbr_utilisateur, inputs=file_input, outputs=out_plot_utilisateur)

    with gr.Tab("Nombre de publications par hashtag"):
        file_input2 = gr.File(label="Charger ZA.json")
        bouton_hashtag = gr.Button("Afficher")
        out_plot_hashtag = gr.Plot()
        bouton_hashtag.click(afficher_nbr_hashtag, inputs=file_input2, outputs=out_plot_hashtag)

    with gr.Tab("Nombre de publications par topic"):
        file_input3 = gr.File(label="Charger ZA.json")
        bouton_topic = gr.Button("Afficher")
        out_plot_topic = gr.Plot()
        bouton_topic.click(afficher_nbr_topic, inputs=file_input3, outputs=out_plot_topic)

    with gr.Tab("Top K Hashtags"):
        file_input4 = gr.File(label="Charger ZA.json")
        k_input = gr.Number(label="K", value=3)
        bouton_top_hashtags = gr.Button("Afficher")
        out_plot_top_hashtags = gr.Plot()
        bouton_top_hashtags.click(afficher_top_hashtags, inputs=[file_input4, k_input], outputs=out_plot_top_hashtags)

    with gr.Tab("Top K Auteurs"):
        file_input5 = gr.File(label="Charger ZA.json")
        k_input2 = gr.Number(label="K", value=3)
        bouton_top_auteur = gr.Button("Afficher")
        out_plot_top_auteur = gr.Plot()
        bouton_top_auteur.click(afficher_top_auteur, inputs=[file_input5, k_input2], outputs=out_plot_top_auteur)

    with gr.Tab("Top K Mentions"):
        file_input6 = gr.File(label="Charger ZA.json")
        k_input3 = gr.Number(label="K", value=3)
        bouton_top_mentions = gr.Button("Afficher")
        out_plot_top_mentions = gr.Plot()
        bouton_top_mentions.click(afficher_top_mentions, inputs=[file_input6, k_input3], outputs=out_plot_top_mentions)

    with gr.Tab("Utilisateurs mentionnant un hashtag"):
        file_input7 = gr.File(label="Charger ZA.json")
        hashtag_input = gr.Textbox(label="Hashtag (ex: #CIV)")
        bouton_mention_hashtag = gr.Button("Afficher")
        out_plot_mention_hashtag = gr.Plot()
        bouton_mention_hashtag.click(afficher_utilisateurs_mentions_hashtag, inputs=[file_input7, hashtag_input], outputs=out_plot_mention_hashtag)

    with gr.Tab("Utilisateurs mentionnant un utilisateur"):
        file_input8 = gr.File(label="Charger ZA.json")
        mention_input = gr.Textbox(label="Utilisateur mentionné (ex: @leonnajulie)")
        bouton_mention_utilisateur = gr.Button("Afficher")
        out_plot_mention_utilisateur = gr.Plot()
        bouton_mention_utilisateur.click(afficher_utilisateur_mentions_utilisateur, inputs=[file_input8, mention_input], outputs=out_plot_mention_utilisateur)

    with gr.Tab("Utilisateurs mentionnés par un utilisateur"):
        file_input9 = gr.File(label="Charger ZA.json")
        user_input = gr.Textbox(label="Auteur (ex: '372993152')")
        bouton_user_mentions = gr.Button("Afficher")
        out_plot_user_mentions = gr.Plot()
        bouton_user_mentions.click(afficher_utilisateurs_mentionnes_par_utilisateur, inputs=[file_input9, user_input], outputs=out_plot_user_mentions)

demo.launch()


* Running on local URL:  http://127.0.0.1:7867

To create a public link, set `share=True` in `launch()`.


In [ ]:
t= "@ericbailly24 @maxigr04del  mes tontons vous avez fait votre part , JO prochain on ira en demi final au moins. BRAVO à vous . #SupportriceMazo #domie #CIV"


def Nettoyage_Tweet(tweet):
    """Fonction qui supprime les caractères spéciaux d'un tweet"""
    enlever_speciaux = re.sub(r'[^a-zA-Z0-9\s@#]', '', tweet)
    return enlever_speciaux

def Extraction_Sentiments(tweet): 
        """Extrait le sentiment du texte"""
        blob = TextBlob(tweet)
        polarite =  blob.sentiment.polarity
        if polarite > 0:
            sentiment = 'positif'
        elif polarite < 0:
            sentiment = 'négatif'
        elif polarite == 0:
            sentiment = 'neutre'
        return sentiment
    
def Extraction_Hashtags(tweet):  #A Faire 
        """"""
        hashtags = re.findall(r'#\w+', tweet)
        return hashtags
    
def Extraction_Mentions(tweet):  #A Faire
        """Fonction qui extrait les mentions de personnes dans un tweet"""
        mentions = re.findall(r'@[A-Za-z0-9_]+', tweet)
        return mentions


print(Nettoyage_Tweet(t))
print(Extraction_Sentiments(t))
print(Extraction_Hashtags(t))
print(Extraction_Mentions(t))




In [13]:
import gradio as gr
import pandas as pd
import os

# Fonctions pour Gradio
def charger_fichier(fichier):
    df = CreerDataFrame(fichier.name)
    return df.head()  # Afficher les 5 premières lignes

def gr_top_k(fichier, colonne, k):
        df = CreerDataFrame(fichier.name)
        elements, counts = Top_K_element(df, colonne, int(k))
        return pd.DataFrame({"Éléments": elements, "Occurrences": counts})


def gr_hashtags(fichier):
    
    df = CreerDataFrame(fichier.name)
    hashtags, counts = publications_par_hashtag(df)
    return pd.DataFrame({"Hashtags": hashtags, "Occurrences": counts})
    
def gr_utilisateurs(fichier):
    
    df = CreerDataFrame(fichier.name)
    utilisateurs, counts = publications_par_utilisateurs(df)
    return pd.DataFrame({"Utilisateurs": utilisateurs, "Occurrences": counts})

def gr_topics(fichier):
    
    df = CreerDataFrame(fichier.name)
    topics, counts = publications_par_topic(df)
    return pd.DataFrame({"Topics": topics, "Occurrences": counts})

def generate_top_k_graph(df, colonne, k=3):
    try:
        # Appeler la fonction Top_K_element pour obtenir les éléments les plus fréquents
        elements, counts = Top_K_element(df, colonne, k)
        
        # Créer un graphique
        fig, ax = plt.subplots(figsize=(6, 4))
        ax.bar(elements, counts, color='purple')
        ax.set_title(f"Top {k} {colonne}")
        ax.set_xlabel(colonne)
        ax.set_ylabel("Fréquence")
        ax.tick_params(axis="x", rotation=50)

        # Sauvegarder l'image dans un fichier
        img_path = os.path.join(graph_sortis, f"top_{colonne}_{k}.png")
        plt.tight_layout()
        plt.savefig(img_path)  # Sauvegarder l'image dans le fichier
        plt.close()  # Fermer pour libérer la mémoire

        return img_path  # Retourner le chemin de l'image générée
    except Exception as e:
        return str(e)

    

def gr_tweets(fichier, utilisateur):

    df = CreerDataFrame(fichier.name)
    utilisateur, tweets = tweets_par_utilisateur(df, utilisateur)
    return {"Utilisateur": utilisateur, "Tweets": tweets}

# Interface Gradio
with gr.Blocks() as demo:
    gr.Markdown("Analyse des données JSON avec Gradio")
    
    with gr.Tab("Charger le fichier"):
        fichier = gr.File(label="Télécharger un fichier JSON")
        bouton_charger = gr.Button("Charger")
        sortie_df = gr.Dataframe()
        bouton_charger.click(charger_fichier, inputs=[fichier], outputs=[sortie_df])

    with gr.Tab("Top K éléments"):
        fichier_top_k = gr.File(label="Télécharger un fichier JSON")
        colonne = gr.Textbox(label="Nom de la colonne", value="hashtags")  # Exemple : hashtags, auteur, etc.
        k = gr.Number(label="K", value=3)  # Par défaut, afficher les 3 premiers éléments
        bouton_top_k = gr.Button("Afficher Top K")
        sortie_top_k_graph = gr.Image(label="Graphique Top K")  # Afficher le graphique ici

        # Lier le bouton pour générer le graphique et l'afficher
        bouton_top_k.click(generate_top_k_graph, inputs=[fichier_top_k, colonne, k], outputs=[sortie_top_k_graph])


    with gr.Tab("Publications par hashtags"):
        fichier = gr.File(label="Télécharger un fichier JSON")
        bouton_hashtags = gr.Button("Afficher les hashtags")
        sortie_hashtags = gr.Dataframe()
        bouton_hashtags.click(gr_hashtags, inputs=[fichier], outputs=[sortie_hashtags])

    with gr.Tab("Publications par utilisateurs"):
        fichier = gr.File(label="Télécharger un fichier JSON")
        bouton_utilisateurs = gr.Button("Afficher les utilisateurs")
        sortie_utilisateurs = gr.Dataframe()
        bouton_utilisateurs.click(gr_utilisateurs, inputs=[fichier], outputs=[sortie_utilisateurs])

    with gr.Tab("Publications par topics"):
        fichier = gr.File(label="Télécharger un fichier JSON")
        bouton_topics = gr.Button("Afficher les topics")
        sortie_topics = gr.Dataframe()
        bouton_topics.click(gr_topics, inputs=[fichier], outputs=[sortie_topics])

    with gr.Tab("Tweets par utilisateur"):
        fichier = gr.File(label="Télécharger un fichier JSON")
        utilisateur = gr.Textbox(label="Nom de l'utilisateur")
        bouton_tweets = gr.Button("Afficher les tweets")
        sortie_tweets = gr.JSON()
        bouton_tweets.click(gr_tweets, inputs=[fichier, utilisateur], outputs=[sortie_tweets])

# Lancer l'application
demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7865
* Running on public URL: https://0e23db567fdf07e35b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/Users/jesse/miniconda3/lib/python3.11/site-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jesse/miniconda3/lib/python3.11/site-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jesse/miniconda3/lib/python3.11/site-packages/gradio/blocks.py", line 2015, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jesse/miniconda3/lib/python3.11/site-packages/gradio/blocks.py", line 1562, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jesse/miniconda3/lib/python3.11/site-packages/anyio/to_thread.py", line 56, in run_sync
    return awai